In [24]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
from glob import glob
from os import getcwd
from os.path import join
import urllib
from collections import Counter

################################################################################
## General Stuff
################################################################################
from searchUtils import findNearest
from searchUtils import findExt
from searchUtils import findPatternExt
from fileUtils import getBasename, getDirname, getBaseFilename
from timeUtils import clock, elapsed
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from listUtils import getFlatList
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from time import sleep

################################################################################
## Database Stuff
################################################################################
from dbBase import dbBase
from mainDB import mainDB
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import masterdb



from multiArtist import multiartist
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-10-27 19:35:25.535393


In [27]:
from os.path import join
from fsUtils import isFile, setFile
from ioUtils import saveFile, getFile
from pandas import DataFrame

class soundOpinions:
    def __init__(self, debug=False):
        self.debug = debug

        self.baseURL = "https://www.soundopinions.org"
        
        self.basename = "soundopinions"
        self.savedir  = "/Volumes/Piggy/Charts/data/{0}".format(self.basename)
        
        self.starterSavename = join(self.savedir, "starter.p")

        self.listsURL = "{0}/section/list".format(self.baseURL)
        self.listsDir = join(self.savedir, "lists")
        self.listsYearlyDir = join(self.savedir, "lists", "yearly")
        self.listsSavename = join(self.savedir, "lists.p")

        #self.showsURL = "{0}/section/list".format(baseURL)
        self.showsDir = join(self.savedir, "shows")
        self.showsSavename = join(self.savedir, "shows.p")

    def downloadURL(self, url):
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        return data, response.getcode()


    def saveDownload(self, url, savename):
        if isFile(savename):
            return

        print("Downloading {0}".format(url))
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False
        
        print("Saving {0}".format(savename))
        saveFile(idata=data, ifile=savename)
        sleep(1)
        
        
        
    def downloadStarterURL(self):
        url      = self.baseURL
        self.saveDownload(url, self.starterSavename)
        
    def parseStarterURL(self):
        data = getHTML(self.starterSavename)
        return data
    
    
    
        
    def downloadListsURL(self):
        url      = self.listsURL
        self.saveDownload(url, self.listsSavename)
        
    def downloadYearlyLists(self):
        data = getHTML(self.listsSavename)
        aside = data.find("aside", {"class": "index"})
        if aside is not None:
            refs = aside.findAll("a")
            for ref in refs:
                name     = ref.text
                url      = "{0}{1}".format(self.baseURL, ref.attrs['href'])
                savename = join(self.listsYearlyDir, "{0}.p".format(name))
                self.saveDownload(url, savename)
                
    def downloadLists(self):
        files = findExt(self.listsYearlyDir, ext='.p')
        for ifile in files:
            data = getHTML(ifile)
            article = data.find("article", {"class": "collection"})
            if article is not None:
                uls = article.findAll("ul")
                for ul in uls:
                    for ul in uls:
                        lis = ul.findAll("li")
                        for li in lis:
                            ref  = li.find('a')
                            name = ref.text
                            url  = "{0}{1}".format(self.baseURL, ref.attrs['href'])
                            savename = setFile(self.listsDir, "{0}.p".format(name))
                            #continue
                            self.saveDownload(url, savename)
                            
                            
    def parseListLine(self, line):
        retvals = {}
        line  = line.replace("<q>", "")
        line  = line.replace("</q>", "")
        line  = line.replace(",,", ",")
        lvals = line.split(",")
        if len(lvals) > 1:        
            if lvals[1] == "000 Maniacs":
                lvals[1] = "10,000 Maniacs"
                lvals = lvals[1:]
        if len(lvals) > 5:
            if lvals[3] == " War and the Ghost of Whitey Ford":
                lvals[3] = "Love, War and the Ghost of Whitey Ford"
                lvals.pop(2)
            if lvals[3] == " Here’s the Sex Pistols":
                lvals[3] = "Never Mind the Bollocks, Here’s the Sex Pistols"
                lvals.pop(2)
        if len(lvals) > 6:        
            if lvals[3] == " Brother" and lvals[4] == " Brother":
                lvals[2] = "Brother, Brother, Brother"
                lvals.pop(3)
                lvals.pop(4)

        if len(lvals) == 5:
            retvals = dict(zip(keys, lvals))
        elif len(lvals) == 1:
            slvals = lvals[0].split(" - ")
            if len(slvals) == 2:
                skeys = ["Artist", "Title"]
                retvals = dict(zip(skeys, slvals))
            else:
                print(line)
                print(len(lvals),'\t',"\t".join(lvals))
        elif len(lvals) == 2:
            skeys = ["Artist", "Title"]
            retvals = dict(zip(skeys, lvals))
        elif len(lvals) == 3:
            skeys = ["Artist", "Title", "Year"]
            retvals = dict(zip(skeys, lvals))
        elif len(lvals) == 4:
            skeys = ["Artist", "Title", "Album", "Year"]
            retvals = dict(zip(skeys, lvals))
        elif len(lvals) == 8:
            skeys = ["Artist", "Title", "Album", "Year"]
            slvals = [lvals[0], lvals[1], lvals[2], lvals[-1]]
            retvals = dict(zip(skeys, slvals))
        else:
            return {}
            print(line)
            print(len(lvals),'\t',"\t".join(lvals))

        return retvals
    
    def getLists(self):
        files   = sorted(findExt(self.listsDir, ext='.p'))
        results = []
        
        for ifile in files:
            data = getHTML(ifile)
            section = data.find("section", {"id": "featured-songs"})
            if section is not None:
                ols = section.findAll("ol")
                for ol in ols:
                    lis = ol.findAll("li")
                    for li in lis:
                        line = li.text
                        retvals = self.parseListLine(line)
                        results.append(retvals)

        df = DataFrame(results)
        df = df[~df["Artist"].isna()]                        
        return df
    
    
    def createCharts(self, idata=None):
        if idata is None:
            idata = getFile("SoundOpinionLists.p")
        artistAlbumData = {}
        for artistName,df in idata.groupby("Artist"):
            titles = list(set(list(df["Title"].values) + list(df["Album"].values)))
            artistAlbumData[artistName] = titles
        saveFile(idata=artistAlbumData, ifile="currentSoundOpinionsArtistAlbumData.p", debug=True)


        fullChartData = {}
        for artistName,df in idata.groupby("Artist"):
            songs  = [str(x) for x in list(df["Title"].values)]
            years  = [str(x) for x in list(df["Year"].values)]
            albums = [str(x) for x in list(df["Album"].values)]

            songData  = {item[0].strip(): {"SoundOpinions": {item[1].strip(): 1}} for item in list(zip(songs, years))}
            albumData = {item[0].strip(): {"SoundOpinions": {item[1].strip(): 1}} for item in list(zip(albums, years))}

            fullChartData[artistName.strip()] = {"Songs": songData, "Albums": albumData}
        saveFile(idata=fullChartData, ifile="currentSoundOpinionsFullChartArtistAlbumData.p", debug=True)

In [28]:
so = soundOpinions()

In [ ]:
so.downloadStarterURL()
so.downloadListsURL()
so.downloadYearlyLists()
so.downloadLists()

In [30]:
#df = so.getLists()
so.createCharts()
#saveFile(idata=df, ifile="SoundOpinionLists.p")

Saving data to currentSoundOpinionsArtistAlbumData.p
  --> This file is 113.4kB.
Saved data to currentSoundOpinionsArtistAlbumData.p
  --> This file is 113.4kB.
Saving data to currentSoundOpinionsFullChartArtistAlbumData.p
  --> This file is 201.7kB.
Saved data to currentSoundOpinionsFullChartArtistAlbumData.p
  --> This file is 201.7kB.
  --> This file is 811B.


In [6]:
idata = getFile("SoundOpinionLists.p")

Saving data to currentSoundOpinionsArtistAlbumData.p
  --> This file is 113.4kB.
Saved data to currentSoundOpinionsArtistAlbumData.p
  --> This file is 113.4kB.


Saving data to currentTop40FullChartArtistAlbumData.p
  --> This file is 201.7kB.
Saved data to currentTop40FullChartArtistAlbumData.p
  --> This file is 201.7kB.


In [ ]:

    <main id="content" class="container">
      <aside class="index">
        <div class="player">
          <!-- @see player.js -->
        </div>
        <nav>
          <a href="/section/list/2020" class="selected">2020</a>
          <a href="/section/list/2019">2019</a>
          <a href="/section/list/2018">2018</a>
          <a href="/section/list/2017">2017</a>
          <a href="/section/list/2016">2016</a>
          <a href="/section/list/2015">2015</a>
          <a href="/section/list/2014">2014</a>
          <a href="/section/list/2013">2013</a>
          <a href="/section/list/2012">2012</a>
          <a href="/section/list/2011">2011</a>
          <a href="/section/list/2010">2010</a>
          <a href="/section/list/2009">2009</a>
          <a href="/section/list/2008">2008</a>
          <a href="/section/list/2007">2007</a>
          <a href="/section/list/2006">2006</a>
          <a href="/section/list/2005">2005</a>
        </nav>
      </aside>        

In [13]:
idata

,Artist,Title,Album,Label,Year
0,Alvvays,In Undertow,Antisocialites,Polyvinyl,2017
1,AC/DC,Riff Raff,Powerage,Atlantic,1978
2,The Smithereens,A Girl Like You,11,Capitol,1989
3,Tommy Keene,Places That Are Gone,Songs from the Film,Geffen,1986
5,Stevie Wonder,My Cherie Amour,My Cherie Amour,Motown,1969
...,...,...,...,...,...
4354,Phish,Time Turns Elastic,Joy,JEMP,2009
4355,Vivian Girls,Tension,Everything Goes Wrong,In the Red,2009
4356,Vivian Girls,The End,Everything Goes Wrong,In the Red,2009
4358,The Pretenders,The Phone Call,Pretenders,Real,1980


In [ ]:
data